<a href="https://colab.research.google.com/github/multikokoiti1998/vgg_8_3/blob/main/VGG16%EF%BC%88GAN)_ipynp_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import time
import os
from tqdm import tqdm
import random
import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torchvision.models as models
import torchvision
from PIL import Image
import torch.nn as nn
import torch
import pandas as pd
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torch.utils.data import Dataset
from pathlib import Path
from typing import List, Tuple
from pathlib import Path
import xml.etree.ElementTree as ET

In [10]:
# 画像ディレクトリと画像パスの取得
image_directory = '/content/drive/MyDrive/lernig/OBI_ref/image'
label_directory = '/content/drive/MyDrive/lernig/OBI_ref/label'

def update_xml_paths(image_directory: str, label_directory: str):
    image_paths = []
    labels = []

    # 画像ファイルとXMLファイルの一致確認
    for image_file in os.listdir(image_directory):
        if image_file.endswith('.jpg'):
            base_name = image_file.replace('.jpg', '')
            print(base_name)
            xml_path = Path(label_directory) / f"{base_name}.xml"
            print(xml_path)
            if xml_path.exists():
                tree = ET.parse(xml_path)
                root = tree.getroot()
                label = root.find('.//name')
                image_paths.append(os.path.join(image_directory, image_file))
                labels.append(label.text)

    return image_paths, labels

image_paths, labels = update_xml_paths(image_directory, label_directory)
print(image_paths)
print(labels)

0-20005
/content/drive/MyDrive/lernig/OBI_ref/label/0-20005.xml
022
/content/drive/MyDrive/lernig/OBI_ref/label/022.xml
019
/content/drive/MyDrive/lernig/OBI_ref/label/019.xml
09
/content/drive/MyDrive/lernig/OBI_ref/label/09.xml
027
/content/drive/MyDrive/lernig/OBI_ref/label/027.xml
0-20006
/content/drive/MyDrive/lernig/OBI_ref/label/0-20006.xml
0-20003
/content/drive/MyDrive/lernig/OBI_ref/label/0-20003.xml
0-20004
/content/drive/MyDrive/lernig/OBI_ref/label/0-20004.xml
020
/content/drive/MyDrive/lernig/OBI_ref/label/020.xml
01
/content/drive/MyDrive/lernig/OBI_ref/label/01.xml
015
/content/drive/MyDrive/lernig/OBI_ref/label/015.xml
04
/content/drive/MyDrive/lernig/OBI_ref/label/04.xml
02
/content/drive/MyDrive/lernig/OBI_ref/label/02.xml
05
/content/drive/MyDrive/lernig/OBI_ref/label/05.xml
018
/content/drive/MyDrive/lernig/OBI_ref/label/018.xml
011
/content/drive/MyDrive/lernig/OBI_ref/label/011.xml
013
/content/drive/MyDrive/lernig/OBI_ref/label/013.xml
023
/content/drive/MyDrive

In [11]:
data_transform = {
    'train': transforms.Compose(
        [transforms.Resize((256, 256)),
         transforms.CenterCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.RandomRotation(degrees=[-15, 15]),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.35187622, 0.35187622, 0.35187622],
                              std=[0.33888655, 0.33888655, 0.33888655]),
         ]),
    'val': transforms.Compose(
        [transforms.Resize((256, 256)),
         transforms.CenterCrop(224),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.35187622, 0.35187622, 0.35187622],
                              std=[0.33888655, 0.33888655, 0.33888655]),
         ])
}



In [13]:
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index]).convert('RGB')
        label = self.labels[index]
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# データセットの分割とデータローダーの作成
for fold, (train_index, val_index) in enumerate(kfold.split(image_paths), 1):
    train_paths = [image_paths[i] for i in train_index]
    train_labels = [labels[i] for i in train_index]
    val_paths = [image_paths[i] for i in val_index]
    val_labels = [labels[i] for i in val_index]

    train_dataset = ImageDataset(train_paths, train_labels, transform=data_transform['train'])
    val_dataset = ImageDataset(val_paths, val_labels, transform=data_transform['val'])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    dataloaders  = {"train":train_loader, "val":val_loader}
# ここでデータローダーを使用して学習や検証を行う
num_epochs=100
train_losses = []
val_losses = []
train_accs = []
val_accs = []
data_size = {'train': len(train_dataset), 'val': len(val_dataset)}
def train_model(model, criterion, optimizer, num_epochs):

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / data_size[phase]
            epoch_acc = corrects.double() / data_size[phase]

            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accs.append(epoch_acc)
                print('{}Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            else:
                val_losses.append(epoch_loss)
                val_accs.append(epoch_acc)
                print('{}Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

          #print('{}Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    train_accs_cpu = [acc.cpu().numpy() for acc in train_accs]
    val_accs_cpu = [acc.cpu().numpy() for acc in val_accs]

    plot_training_history(train_losses, val_losses, train_accs_cpu, val_accs_cpu)



def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    epochs = range(1, len(train_losses) + 1)

    # Lossのプロット
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'bo-', label='Training Loss')
    plt.plot(epochs, val_losses, 'ro-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracyのプロット
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accs, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_accs, 'ro-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.ylim(0.4, 1)
    plt.tight_layout()
    plt.show()



In [ ]:
import torchvision.models as models
import torch.optim as optim

# ResNet18の事前学習済みモデルをロード#resnet以外も
net = torchvision.models.alexnet(pretrained=True)

for param in net.parameters():
    param.requires_grad = False

# 事前学習済みモデルの全結合層の出力ユニット数をデータのクラス数に置き換え
num_classes = len(train_dataset.classes)
net.classifier[6] = nn.Linear(4096, num_classes)

# モデルをGPUに移動

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

# 損失関数と最適化アルゴリズムを定義(MSE）
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

# モデルのトレーニングと評価
model = net
base_path = r'/content/drive/MyDrive/learning'
folder_name = 'weight'
file_name = 'alexnet.pth'
file_path = os.path.join(base_path, folder_name, file_name)


train_model(net, criterion, optimizer, num_epochs)
print('Train done.')
 # 全エポックの平均損失と平均精度を計算
average_train_loss = sum(train_losses) / num_epochs
average_train_acc = sum(train_accs) / num_epochs
average_val_loss = sum(val_losses) / num_epochs
average_val_acc = sum(val_accs) / num_epochs
average_train_acc_cpu = average_train_acc.cpu().numpy()
average_val_acc_cpu = average_val_acc.cpu().numpy()
# 結果を表示
print("Average Training Loss:", average_train_loss)
print("Average Training Accuracy:", average_train_acc_cpu)
print("Average Validation Loss:", average_val_loss)
print("Average Validation Accuracy:", average_val_acc_cpu)

torch.save(model.state_dict(), file_path)
print('Train saved.')

In [ ]:
model.load_state_dict(torch.load(r'/content/drive/MyDrive/learning/weight/alexnet.pth'))
print('Train loaded.')